In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from darts import TimeSeries
from darts.metrics import mql
from darts.models.forecasting.linear_regression_model import LinearRegressionModel
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting.probabilistic import (
    PinballLoss,
)
from sktime.split import (
    ExpandingWindowSplitter,
    temporal_train_test_split,
)

from probafcst.models import BenchmarkForecaster
from probafcst.plotting import plot_quantiles

sns.set_theme(style="ticks")

In [ ]:
bikes = pd.read_parquet("../data/bikes.parquet")
load = pd.read_parquet("../data/energy.parquet")

In [ ]:
ts = TimeSeries.from_series(bikes.bike_count)

quantile_levels = [0.025, 0.25, 0.5, 0.75, 0.975]
train, test = ts.split_after(0.95)
forecaster = LinearRegressionModel(
    lags=[-24, -24 * 7],
    likelihood="quantile",
    quantiles=quantile_levels,
    multi_models=False,
    output_chunk_length=len(test),
    add_encoders={
        "cyclic": {"future": ["month"]},
        "datetime_attribute": {"future": ["dayofmonth", "dayofweek"]},
    },
)
forecaster.fit(train)

In [ ]:
y_pred = forecaster.predict(
    n=len(test), series=train, predict_likelihood_parameters=True
)

test.plot()
y_pred.plot()

In [ ]:
mql(test, y_pred, q=quantile_levels).mean()

In [ ]:
splitter = ExpandingWindowSplitter(
    fh=ForecastingHorizon(np.arange(1, 24 * 7)),
    initial_window=24 * 365 * 3,  # 3 years
    step_length=24 * 30 * 3,
)
len(list(splitter.split(load)))

In [ ]:
y_train, y_test = temporal_train_test_split(
    load.iloc[-24 * 365 * 3 :], test_size=24 * 14
)
print(len(y_train), len(y_test))
fh = ForecastingHorizon(y_test.index, is_relative=False)
forecaster = BenchmarkForecaster(n_weeks=75)
forecaster.fit(y_train)

In [ ]:
pred_quantiles = forecaster.predict_quantiles(fh, alpha=quantile_levels)
plot_quantiles(y_test, pred_quantiles)

In [ ]:
results = evaluate(
    forecaster,
    splitter,
    load,
    strategy="refit",
    scoring=PinballLoss(score_average=False, alpha=quantile_levels),
    return_data=True,
)
results.head()

In [ ]:
# each entry in test_PinballLoss is a Series with the quantile levels as index
# expand them into columns
results = pd.concat(
    [
        results.drop(columns="test_PinballLoss"),
        results["test_PinballLoss"].apply(pd.Series),
    ],
    axis=1,
)
results

In [ ]:
results[quantile_levels].mean(axis=0)

In [ ]:
results[quantile_levels].mean(axis=None)

In [ ]:
results.iloc[:, :-3]

In [ ]:
# plot each forecast period
nrows = min(3, len(results))
for i, row in results.iloc[-nrows:].iterrows():
    plot_quantiles(row.y_test, row.y_pred_quantiles)